# Analysis

Training self-made models. For transfer-learning, see `transfer.ipynb`.

# Colab Setup

1. link to and refresh Google Drive
1. Set `pwd` to `colab_notebooks/code/` and import **tensorflow**
1. Check if low or high RAM runtime
1. Connect Tensorflow to Hardware Accelerator, either
  1. GPU, or
  1. TPU

In [ ]:
# 1
from google.colab import drive
#drive.mount('/content/drive')
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# 2
!pwd
%cd drive/MyDrive/colab_notebooks/code/

import sys
import os
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

/content
/content/drive/MyDrive/colab_notebooks/code
Tensorflow version 2.3.0


In [ ]:
# 3 
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
# 4.1 GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

device_name = tf.test.gpu_device_name()
print(device_name)
tf.device(device_name)

Tue Dec 15 01:48:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# 4.2 TPU
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
#tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

# CNN 2D Deep 

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

from data_mgmt import make_2D_datasets

In [ ]:
batch_size = 32
img_hw = 160
root_dir = 'data'

In [ ]:
[train_ds, test_ds] = make_2D_datasets(batch_size, img_hw, root_dir)

Found 147358 files belonging to 2 classes.
Found 37738 files belonging to 2 classes.


In [8]:
inputs = keras.Input(shape=(img_hw, img_hw, 3,)) # omit specifying batch size explicitly
x = layers.experimental.preprocessing.Rescaling(scale=1./127.5, offset=-1)(inputs) # or just 1./255
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(48, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(48, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(48, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(80, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(80, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(80, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(96, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(96, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(96, 3, activation='relu', padding='same')(x)
x = layers.GlobalAveragePooling2D()(x)
#x = layers.Flatten()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name='cnn2d_deep_1')

base_learning_rate = 0.000005

model.compile(
    #loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
    metrics=['accuracy', 'AUC'],
)

model.summary()

Model: "cnn2d_deep_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
rescaling_4 (Rescaling)      (None, 160, 160, 3)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 160, 160, 32)      896       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 160, 160, 32)      9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 80, 80, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 80, 80, 1)         289       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 80, 80, 32)       

In [ ]:
checkpoint_path = '../saved_models/checkpoints/cnn2D_deep_1.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cnn2D_deep_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

n_epochs=10

history = model.fit(train_ds, epochs=n_epochs, verbose=1,
                    validation_data=test_ds,
                    callbacks = [cnn2D_deep_callback])

model.save('../saved_models/'+'cnn2d_deep_1')

Epoch 1/10
  29/4605 [..............................] - ETA: 19:36:35 - loss: 0.6926 - accuracy: 0.8351 - auc: 0.5000

In [ ]:
import csv
from datetime import datetime

loss = history.history['loss']
accr = history.history['accuracy']
auc = history.history['auc']
val_l = history.history['val_loss']
val_a = history.history['val_accuracy']
val_auc=history.history['val_auc']
file_contents = np.array([loss, accr, auc, val_l, val_a, val_auc]).T
filename = '../visualization/plotting_data/'+'cnn2d_deep_1'+'_training_history.csv'
file = open(filename, 'w+', newline ='')   
with file:
    write = csv.writer(file)
    write.writerows(file_contents)

# ResNet 3D


In [5]:
from tensorflow import keras
from tensorflow.keras import layers

from data_mgmt import make_generators

In [6]:
batch_size = 32
img_hw = 160
n_frames = 16
root_dir = 'data'

[training_generator, testing_generator] = make_generators(batch_size, img_hw, n_frames, root_dir)
    
input_shape = (n_frames, img_hw, img_hw, 3)

inputs = keras.Input(shape=input_shape)
x = layers.experimental.preprocessing.Rescaling(scale=1./127.5, offset=-1)(inputs)
x = layers.Conv3D(32, 3, activation='relu')(x)
x = layers.Conv3D(64, 3, activation='relu')(x)
block_1_output = layers.MaxPooling3D(3)(x)

x = layers.Conv3D(64, 3, activation='relu', padding='same')(block_1_output)
x = layers.Conv3D(64, 3, activation='relu', padding='same')(x)
block_2_output = layers.add([x, block_1_output])

x = layers.Conv3D(64, 3, activation='relu', padding='same')(block_2_output)
x = layers.Conv3D(64, 3, activation='relu', padding='same')(x)
block_3_output = layers.add([x, block_2_output])

x = layers.Conv3D(128, 3, activation='relu')(block_3_output)
x = layers.GlobalAveragePooling3D()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs, name='resnet3D_1')

base_learning_rate = 0.000005
model.compile(
  optimizer=tf.keras.optimizers.SGD(lr=base_learning_rate, momentum=0.9), 
  #loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
  metrics=['accuracy', 'AUC'])


model.summary()

Model: "resnet3D_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 16, 160, 160 0                                            
__________________________________________________________________________________________________
rescaling_2 (Rescaling)         (None, 16, 160, 160, 0           input_3[0][0]                    
__________________________________________________________________________________________________
conv3d_14 (Conv3D)              (None, 14, 158, 158, 2624        rescaling_2[0][0]                
__________________________________________________________________________________________________
conv3d_15 (Conv3D)              (None, 12, 156, 156, 55360       conv3d_14[0][0]                  
_________________________________________________________________________________________

In [ ]:
checkpoint_path = '../saved_models/checkpoints/resnet3D_1.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

resnet3D_1_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

n_epochs=10

history = model.fit(train_ds, epochs=n_epochs, verbose=1,
                    validation_data=test_ds,
                    callbacks = [resnet3D_1_callback])

model.save('../saved_models/'+'resnet3D_1')

In [ ]:
import csv
from datetime import datetime

loss = history.history['loss']
accr = history.history['accuracy']
auc = history.history['auc']
val_l = history.history['val_loss']
val_a = history.history['val_accuracy']
val_auc=history.history['val_auc']
file_contents = np.array([loss, accr, auc, val_l, val_a, val_auc]).T
filename = '../visualization/plotting_data/'+'resnet3D_1'+'_training_history.csv'
file = open(filename, 'w+', newline ='')   
with file:
    write = csv.writer(file)
    write.writerows(file_contents)

# iD3 aka RD3 Self Made

Only if you have time; already used pre-trained version in transfer learning.

In [ ]:
from data_mgmt import make_generators
from models3D import hara

n_epoch = 5
batch_size=20
img_hw =160
n_frames=16

root_dir = 'small_local'
hara_m = hara(n_epoch, batch_size, img_hw, n_frames, root_dir, True)